In [ ]:
# This notebooks looks at primary mirror motion as a function of elevation 
# This is used to check against mirror motion
# this notebook is really meant to test the new LUT that was derived 
# m1_hex_20210810_v3.yaml:v0.8.2.alpha.2-0-g72c6e32

# PI: 2021-08-25 - it's unclear what this notebook was used for.
# It appears something odd happened.

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient, rendezvous_dataframes

In [ ]:
%matplotlib inline

In [ ]:
#efd_client = EfdClient('summit_efd')
efd_client = EfdClient('ldf_stable_efd') 

Query for all the `endReadout` events on the timespan of the night, elevation, and pmd data

In [ ]:
date='20210812'
test='m1_pressure_reduction_closed_dome'
run='initial'

if run == 'initial': 
# Starting Test1, Nominal LUT from m1_hex_20210810_v3.yaml:v0.8.2.alpha.2-0-g72c6e32
# start time is 2021-08-12T21:01:52.999
    t1 = Time("2021-08-12T21:01:52.999", format='isot', scale='tai')
    t2 = Time("2021-08-12T21:08:33.495", format='isot', scale='tai')
    scale_factor=1.0
    start_log_msg = '[2021-07-08_Repeat_Focus_Test_START]'
    finish_log_msg = '[2021-07-08_Repeat_Focus_Test_END]'
elif run == 'test2':
# Starting Test2, LUT offset by 1211.6539141845715 and is now 122385.77740478516
# start time is 2021-08-12T21:21:00.763
    t1 = Time("2021-08-12T21:21:00.763", format='isot', scale='tai')
    t2 = Time("2021-08-12T21:28:08.745", format='isot', scale='tai')
    scale_factor=1.01
else:
    raise IOError('Not a valid input')

In [ ]:
# # # day of 2021-07-08 - Test3 run with scalefactor = 1.02  - not sure this was great, so redid it
# t1 = Time("2021-08-09T16:42:32.166", format='isot', scale='tai')
# t2 = Time("2021-08-09T16:48:41.015", format='isot', scale='tai')

In [ ]:
# # # day of 2021-07-08 - Test4 run with scalefactor = 1.04 - ignore, redoing
# t1 = Time("2021-08-09T16:50:24.872", format='isot', scale='tai')
# t2 = Time("2021-08-09T16:59:24.872", format='isot', scale='tai')

In [ ]:
# # day of 2021-07-08 - Test4 (didn't increment) run with scalefactor = 1.06 = BAD - Sensor dropped out
# t1 = Time("2021-08-09T16:59:24.872", format='isot', scale='tai')
# t2 = Time("2021-08-09T17:05:32.895", format='isot', scale='tai')

In [ ]:
base0 = await efd_client.select_time_series("lsst.sal.Script.logevent_logMessage", 
                                           ["message","level"], t1, t2)
base0

In [ ]:
t1

In [ ]:
base=base0

In [ ]:
el = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle1", ], t1, t2)
# mount reporting incorrect timestamp for the packed time series
el.index=el.index+pd.tseries.offsets.DateOffset(seconds=-37)

In [ ]:
el = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle1", ], t1, t2)

In [ ]:
pmd = await efd_client.select_time_series("lsst.sal.PMD.position", ["position0", "position1", "position2", "position3", "position4", "position5"], t1, t2)

In [ ]:
pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure", ["pressure"], t1, t2)

In [ ]:
#base

In [ ]:
tmp = rendezvous_dataframes(base, el)

In [ ]:
tmp2= rendezvous_dataframes(tmp, pmd)
tmp3= rendezvous_dataframes(tmp2, pressure)
raw_data = tmp3

In [ ]:
position0_offset = (raw_data.position0[0])
position1_offset = (raw_data.position1[0])
position2_offset = (raw_data.position2[0])
position3_offset = (raw_data.position3[0])
position5_offset = (raw_data.position5[0]) # gauge got moved to position 5

In [ ]:
raw_data=raw_data.drop_duplicates(subset=['elevationCalculatedAngle1','message','pressure'])
print(raw_data[['elevationCalculatedAngle1','pressure']])

In [ ]:
raw_data

In [ ]:
# from scipy import linalg
arr_len = len(raw_data.position0)
coeff_arr = np.zeros((arr_len,3))
theta_arr = np.zeros((arr_len))
phi_arr = np.zeros((arr_len))
piston_arr = np.zeros((arr_len))
for i in np.arange(arr_len):
    # X, Y, Z
    set2=np.array((  41.0, 468.0, (raw_data.position2[i]-position2_offset)))
    set3=np.array(( 384.0,-269.0, (raw_data.position3[i]-position3_offset)))
    set4=np.array((-425.0,-198.0, (raw_data.position5[i]-position5_offset)))

    # Vector PQ crossed with Vector PR
    normal = np.cross(set3-set2,set4-set2) # gives a,b,c

    D= -normal[0]*set2[0] - normal[1]*set2[1] - normal[2]*set2[2]  # Constant in plane equation
    # equation 
    
    phi_from_normal = (np.pi/2+np.arctan2(normal[2], normal[1])) * 206265
    theta_from_normal = (np.pi/2+np.arctan2(normal[2], normal[0])) * 206265
    # find z at the origin to represent piston
    Z_origin = -D/normal[2]
    
    # Measure rotation about the Y-axis (perpendicular to elevation)
    # So this is TIP and should result in motion in azimuth
    # get slope by looking at Y=0, X=400
    x_pt=400; y_pt=0
    #Z_at_x_pt= C[0]*x_pt + C[1]*0.0 + C[2]
    Z_at_x_pt= (-D - normal[0]*x_pt - normal[1]*0.0)/normal[2]
    theta = np.arctan2(Z_at_x_pt-Z_origin, x_pt) * 206265 # arcsec
        
    # Measure rotation about the X-axis (aligned to elevation)
    # this is TILT and should result in motion in elevation
    # get slope by looking at Y=0, X=400    
    x_pt=0; y_pt=400
    Z_at_y_pt= (-D - normal[0]*x_pt - normal[1]*y_pt)/normal[2]
    phi = np.arctan2(Z_at_y_pt-Z_origin, y_pt) * 206265 # arcsec

    theta_arr[i] = theta # arcsec
    phi_arr[i] = phi    # arcsec
    piston_arr[i] = Z_origin
    
#     print(f'theta_from_normal is {theta_from_normal:0.2f}, phi_from_normal is {phi_from_normal:0.2f}')
#     print(f'theta_arr is {theta_arr[i]:0.2f} [arcsec], phi_arr is {phi_arr[i]:0.2f} [arcsec]')

In [ ]:
import copy
calc_data=copy.copy(raw_data)
calc_data['m1_tip']=theta_arr
calc_data['m1_tilt']=phi_arr
calc_data['m1_piston']=piston_arr

calc_data['m1_y_pos']=(raw_data.position0-position0_offset)*np.cos(10*np.pi/180)
calc_data['m1_x_pos']=(raw_data.position1-position1_offset)*np.cos(10*np.pi/180)

In [ ]:
calc_data

In [ ]:
#write data to disk
filename="data/"+date+'_'+test+"_"+run+"_metadata.csv"
calc_data.to_csv(filename)

In [ ]:
fig_height=5
fig_width=10
nwide=2; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')
xvals = (calc_data.m1_x_pos) # um
yvals = calc_data.elevationCalculatedAngle1

row1[0].plot(xvals, yvals, 'o-')
row1[0].set_ylabel('Elevation [deg]')
row1[0].set_xlabel('M1 X-position [mm]')

xvals = (calc_data.m1_tip) # um
row1[1].plot(xvals, yvals, 'o-')
row1[1].set_xlabel('M1 tip [arcsec]')

xvals = (calc_data.m1_y_pos) # mm
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_ylabel('Elevation [deg]')
row2[0].set_xlabel('M1 X-position [mm]')

xvals = (calc_data.m1_tilt) # um
row2[1].plot(xvals, yvals, 'o-')
row2[1].set_xlabel('M1 tilt [arcsec]')

xvals = (calc_data.m1_piston) # mm
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_ylabel('Elevation [deg]')
row3[0].set_xlabel('M1 Z-position [mm]')

xvals = (calc_data.pressure) # mm
row3[1].plot(xvals, yvals, 'o-')
row3[1].set_ylabel('Elevation [deg]')
row3[1].set_xlabel('Pressure [Pa]')

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     hex_elevation